In [261]:
import numpy as np
import pandas as pd

import xgboost as xgb


In [262]:
train_data = pd.read_csv("./train_LZdllcl.csv", index_col= "employee_id")
train_data.head()


,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [263]:
train_data.columns

Index(['department', 'region', 'education', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'previous_year_rating', 'length_of_service',
       'KPIs_met >80%', 'awards_won?', 'avg_training_score', 'is_promoted'],
      dtype='object')

In [264]:
train_data["KPIs_met >80%"].value_counts()

0    35517
1    19291
Name: KPIs_met >80%, dtype: int64

In [265]:
train_data["is_promoted"].value_counts()

0    50140
1     4668
Name: is_promoted, dtype: int64

In [266]:
train_data[train_data["KPIs_met >80%"] == 1].head(200)

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
28911,Sales & Marketing,region_1,Master's & above,m,sourcing,1,32,5.0,5,1,0,54,0
49017,Sales & Marketing,region_7,Bachelor's,f,sourcing,1,35,5.0,3,1,0,50,1
60051,Sales & Marketing,region_4,Bachelor's,m,sourcing,1,49,5.0,5,1,0,49,0
7152,Technology,region_2,Bachelor's,m,other,1,38,3.0,5,1,0,77,0
17436,Sales & Marketing,region_31,Bachelor's,m,other,1,34,5.0,8,1,0,46,0
26585,Technology,region_22,Bachelor's,m,other,1,27,5.0,3,1,0,83,0
37749,Procurement,region_26,Bachelor's,m,other,1,31,5.0,4,1,0,68,0
8219,Analytics,region_5,Bachelor's,m,other,1,29,5.0,3,1,0,80,0


In [267]:
test_data = pd.read_csv("./test_2umaH9m.csv", index_col= "employee_id")
test_data.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
employee_id,,,,,,,,,,,,
8724,Technology,region_26,Bachelor's,m,sourcing,1,24,NaN,1,1,0,77
74430,HR,region_4,Bachelor's,f,other,1,31,3.0,5,0,0,51
72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.0,4,0,0,47
38562,Procurement,region_2,Bachelor's,f,other,3,31,2.0,9,0,0,65
64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.0,7,0,0,61


In [268]:
sample_data = pd.read_csv("./sample_submission_M0L0uXE.csv")
sample_data.head()

,employee_id,is_promoted
0,8724,0
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [269]:
print(train_data.shape, test_data.shape)

(54808, 13) (23490, 12)


In [270]:
train_data.isnull().sum()

department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [271]:
train_data.education.value_counts()

Bachelor's          36669
Master's & above    14925
Below Secondary       805
Name: education, dtype: int64

In [272]:
train_data.previous_year_rating.value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
Name: previous_year_rating, dtype: int64

In [273]:
train_data["education"] = train_data.education.map({"Bachelor's" : 2, "Master's & above": 3, "Below Secondary": 1})
train_data["education"] = train_data["education"].fillna(2)
train_data.head(10)

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,3.0,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,2.0,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,2.0,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,2.0,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,2.0,m,other,1,45,3.0,2,0,0,73,0
58896,Analytics,region_2,2.0,m,sourcing,2,31,3.0,7,0,0,85,0
20379,Operations,region_20,2.0,f,other,1,31,3.0,5,0,0,59,0
16290,Operations,region_34,3.0,m,sourcing,1,33,3.0,6,0,0,63,0
73202,Analytics,region_20,2.0,m,other,1,28,4.0,5,0,0,83,0


In [274]:
# train_data["education"] = train_data["education"].fillna("Bachelor's")
# train_data.head(10)

In [275]:
train_data.previous_year_rating.value_counts()

3.0    18618
5.0    11741
4.0     9877
1.0     6223
2.0     4225
Name: previous_year_rating, dtype: int64

In [276]:
train_data.previous_year_rating.median()

3.0

In [277]:
train_data["previous_year_rating"] = train_data["previous_year_rating"].fillna(train_data.previous_year_rating.median())
train_data.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
employee_id,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,3.0,f,sourcing,1,35,5.0,8,1,0,49,0
65141,Operations,region_22,2.0,m,other,1,30,5.0,4,0,0,60,0
7513,Sales & Marketing,region_19,2.0,m,sourcing,1,34,3.0,7,0,0,50,0
2542,Sales & Marketing,region_23,2.0,m,other,2,39,1.0,10,0,0,50,0
48945,Technology,region_26,2.0,m,other,1,45,3.0,2,0,0,73,0


In [278]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()

In [279]:
# train_data_ohe = pd.get_dummies(train_data)
train_data_ohe = pd.get_dummies(train_data.drop(["region", "recruitment_channel"], axis = 1))
train_data_ohe.shape

(54808, 20)

In [280]:
train_data_ohe.head()

,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted,department_Analytics,department_Finance,department_HR,department_Legal,department_Operations,department_Procurement,department_R&D,department_Sales & Marketing,department_Technology,gender_f,gender_m
employee_id,,,,,,,,,,,,,,,,,,,,
65438,3.0,1,35,5.0,8,1,0,49,0,0,0,0,0,0,0,0,1,0,1,0
65141,2.0,1,30,5.0,4,0,0,60,0,0,0,0,0,1,0,0,0,0,0,1
7513,2.0,1,34,3.0,7,0,0,50,0,0,0,0,0,0,0,0,1,0,0,1
2542,2.0,2,39,1.0,10,0,0,50,0,0,0,0,0,0,0,0,1,0,0,1
48945,2.0,1,45,3.0,2,0,0,73,0,0,0,0,0,0,0,0,0,1,0,1


In [281]:
train_data_ohe.columns

Index(['education', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted', 'department_Analytics',
       'department_Finance', 'department_HR', 'department_Legal',
       'department_Operations', 'department_Procurement', 'department_R&D',
       'department_Sales & Marketing', 'department_Technology', 'gender_f',
       'gender_m'],
      dtype='object')

In [282]:
train_data_ohe.isnull().sum()

education                       0
no_of_trainings                 0
age                             0
previous_year_rating            0
length_of_service               0
KPIs_met >80%                   0
awards_won?                     0
avg_training_score              0
is_promoted                     0
department_Analytics            0
department_Finance              0
department_HR                   0
department_Legal                0
department_Operations           0
department_Procurement          0
department_R&D                  0
department_Sales & Marketing    0
department_Technology           0
gender_f                        0
gender_m                        0
dtype: int64

In [283]:
test_data["education"] = test_data.education.map({"Bachelor's" : 2, "Master's & above": 3, "Below Secondary": 1})
test_data["education"] = test_data["education"].fillna(2)
# test_data["education"] = test_data["education"].fillna("Bachelor's")
test_data["previous_year_rating"] = test_data["previous_year_rating"].fillna(test_data.previous_year_rating.median())


In [284]:
# for feature in ["department", "region", "gender", "recruitment_channel"]:
#     train_data[feature] = le.fit_transform(train_data[feature])
#     test_data[feature] = le.transform(test_data[feature])
    
# train_data.head()

In [285]:
# test_data_ohe = pd.get_dummies(test_data)  
test_data_ohe = pd.get_dummies(test_data.drop(["region", "recruitment_channel"], axis = 1)) 
test_data_ohe.shape

(23490, 19)

In [286]:
test_data_ohe.isnull().sum()

education                       0
no_of_trainings                 0
age                             0
previous_year_rating            0
length_of_service               0
KPIs_met >80%                   0
awards_won?                     0
avg_training_score              0
department_Analytics            0
department_Finance              0
department_HR                   0
department_Legal                0
department_Operations           0
department_Procurement          0
department_R&D                  0
department_Sales & Marketing    0
department_Technology           0
gender_f                        0
gender_m                        0
dtype: int64

In [287]:
features_train = train_data_ohe.drop("is_promoted", axis = 1)
label_train = train_data_ohe.is_promoted

In [288]:
# features_train = train_data.drop("is_promoted", axis = 1)
# label_train = train_data.is_promoted

In [289]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics



X_train, X_test, y_train, y_test = train_test_split(features_train, label_train, test_size = 0.3, random_state = 0)

clfA = RandomForestClassifier(random_state=100)
clfB = LogisticRegression(random_state=0)
clfC = xgb.XGBClassifier(learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

for clf in [clfA, clfB, clfC]:
    clf.fit(X_train, y_train)
    print(metrics.f1_score(y_test, clf.predict(X_test)))

0.47486835806606037
0.36259323006310956
0.5197850512945774


D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [290]:
# from sklearn.model_selection import GridSearchCV


# params = {'n_estimators': [50,100,200,500,1000] }

# grid_obj = GridSearchCV(RandomForestClassifier(random_state=100), params, scoring = 'f1').fit(features_train, label_train)

# print(grid_obj.best_estimator_ , grid_obj.best_score_)

In [291]:
# from sklearn.model_selection import GridSearchCV


# params = {'min_samples_leaf': [1,5,10,20,30,50,100] }

# grid_obj1 = GridSearchCV(RandomForestClassifier(random_state=100, n_estimators=100), params, scoring = 'f1').fit(features_train, label_train)

# print(grid_obj1.best_estimator_ , grid_obj1.best_score_)

In [292]:
# from sklearn.model_selection import GridSearchCV


# params = {'min_samples_split': [2,10,20,30,50], 'max_features': ['auto','sqrt'] }

# grid_obj2 = GridSearchCV(RandomForestClassifier(random_state=100, n_estimators=100), params, scoring = 'f1').fit(features_train, label_train)

# print(grid_obj2.best_estimator_ , grid_obj2.best_score_)

In [293]:
# from sklearn.model_selection import GridSearchCV


# params = {'max_depth': [10, 20, 30, 40, 50], 'bootstrap': [True, False] }

# grid_obj3 = GridSearchCV(RandomForestClassifier(random_state=100, n_estimators=100, min_samples_split=10), params, scoring = 'f1').fit(features_train, label_train)

# print(grid_obj3.best_estimator_ , grid_obj3.best_score_)

In [294]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'max_depth': range(3, 10, 2),
#               'min_child_weight': range(1,6,2),
#               }

# grid_obj4 = GridSearchCV(clfC, parameters,                   
#                    scoring='f1',
#                    verbose=2, n_jobs=4).fit(features_train, label_train)

# print(grid_obj4.best_estimator_ , grid_obj4.best_score_)


In [295]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'gamma':[i/10.0 for i in range(0,5)]
#               }

# grid_clf = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
#        max_depth=5, min_child_weight=5, missing=None, n_estimators=1000,
#        n_jobs=1, nthread=4, objective='binary:logistic', random_state=0,
#        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=27, silent=True,
#        subsample=0.8).fit(features_train, label_train)

# grid_obj5 = GridSearchCV(grid_clf, parameters,                   
#                    scoring='f1',
#                    verbose=2, n_jobs=4).fit(features_train, label_train)

# print(grid_obj5.best_estimator_ , grid_obj5.best_score_)

In [296]:
# from sklearn.model_selection import GridSearchCV

# parameters = {'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
#               }

# grid_clf = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#        colsample_bytree=0.8, gamma=0.1, learning_rate=0.1,
#        max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
#        n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
#        random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
#        seed=27, silent=True, subsample=0.8).fit(features_train, label_train)

# grid_obj6 = GridSearchCV(grid_clf, parameters,                   
#                    scoring='f1',
#                    verbose=2, n_jobs=4).fit(features_train, label_train)

# print(grid_obj6.best_estimator_ , grid_obj6.best_score_)

In [297]:
# clf_final = RandomForestClassifier(random_state=100, n_estimators=100, min_samples_split=10, max_depth=40).fit(features_train, label_train)

clf_final = xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0.1, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=1000, n_jobs=1, nthread=4, objective='binary:logistic',
       random_state=0, reg_alpha=1, reg_lambda=1, scale_pos_weight=1,
       seed=27, silent=True, subsample=0.8).fit(features_train, label_train)

# clf_final = clfC.fit(features_train, label_train)
                              
test_data_ohe["is_promoted"] = clf_final.predict(test_data_ohe)

# test_data["is_promoted"] = clf_final.predict(test_data)

result = test_data_ohe[["is_promoted"]]

result.to_csv("./wns_submit.csv")

D:\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [298]:
# print(sorted(zip(map(lambda x: round(x, 4), clf_final.feature_importances_), train_data_ohe.columns), 
#              reverse=True))